In [1]:
import flopy, os
import sys
import platform
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import flopy.utils.binaryfile as bf

flopy is installed in C:\Users\LENOVO\miniconda3\lib\site-packages\flopy


In [2]:
# Directorio de trabajo:
model_ws= "../../Outputs/Resultados_Base/"
modelname= 'RUCM_Base'
exe_name= '../../Solver/mf2005.exe'
zb_name = 'ZoneBudget_Base'
ml= flopy.modflow.Modflow.load(modelname+'.nam', exe_name=exe_name, 
                               model_ws=model_ws, check=False)

In [3]:
from flopy.utils.zonbud import ZoneBudget, read_zbarray
zon = read_zbarray(os.path.join(model_ws + '/' + modelname + '.zon'))
zb = ZoneBudget(model_ws + '/' + modelname + '.cbc', zon, kstpkper=None)
df = zb.get_dataframes()

## *1. ZONEBUDGET TOTAL (m3/d)* 

In [4]:
# Arreglo del dataframe Zonebudget:
time = ml.dis.get_totim()
nper = ml.dis.nper
entrada = []
salida = []
balance = []
error = []
discrep = []
for i in time:
    entrada.append(df.loc[(i,'TOTAL_IN'), ['ZONE_1','ZONE_2','ZONE_3','ZONE_4',
                                           'ZONE_5','ZONE_6']])
    salida.append(df.loc[(i,'TOTAL_OUT'), ['ZONE_1','ZONE_2','ZONE_3','ZONE_4',
                                           'ZONE_5','ZONE_6']])
    error.append(df.loc[(i,'PERCENT_DISCREPANCY'), ['ZONE_1','ZONE_2','ZONE_3','ZONE_4',
                                           'ZONE_5','ZONE_6']])
for i in range(len(entrada)):
    balance.append(entrada[i]-salida[i])
    discrep.append((entrada[i]-salida[i])/entrada[i]*100)

arrays_inout = [time, np.array(['IN-OUT']*nper)]
df1 = pd.DataFrame(balance, index=arrays_inout)

arrays_discrep = [time, np.array(['PERCENT_DISCREPANCY']*nper)]
df2 = pd.DataFrame(discrep, index=arrays_discrep)

# Reemplazando en df:
for i in time:
    df.loc[(i,'IN-OUT'),'ZONE_1'] = df1.loc[(i,'IN-OUT'),'ZONE_1']
    df.loc[(i,'IN-OUT'),'ZONE_2'] = df1.loc[(i,'IN-OUT'),'ZONE_2']
    df.loc[(i,'IN-OUT'),'ZONE_3'] = df1.loc[(i,'IN-OUT'),'ZONE_3']
    df.loc[(i,'IN-OUT'),'ZONE_4'] = df1.loc[(i,'IN-OUT'),'ZONE_4']
    df.loc[(i,'IN-OUT'),'ZONE_5'] = df1.loc[(i,'IN-OUT'),'ZONE_5']
    df.loc[(i,'IN-OUT'),'ZONE_6'] = df1.loc[(i,'IN-OUT'),'ZONE_6']

for i in time:
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_1'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_1']
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_2'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_2']
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_3'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_3']
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_4'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_4']
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_5'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_5']
    df.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_6'] = df2.loc[(i,'PERCENT_DISCREPANCY'),'ZONE_6']
df.drop(['ZONE_0'], axis = 'columns', inplace=True) # eliminando la columna zona 0

## *2. ZONEBUDGET POR ZONAS* 

In [5]:
perlen = []
for i in range(nper):
    perlen.append(ml.dis.perlen[i])

Years = []
anio_hidro1 = 1980
anio_hidro2 = 81
for i in range(int(len(perlen)/12)):
    if anio_hidro1 < 1999:
        Years.append(str(anio_hidro1)+'/'+str(anio_hidro2))
    else:
        if anio_hidro1 < 2009:
            Years.append(str(anio_hidro1)+'/'+ str(0)+str(anio_hidro2-100))
        else:
            Years.append(str(anio_hidro1)+'/'+str(anio_hidro2-100))
       
    anio_hidro1 = anio_hidro1 +1
    anio_hidro2 = anio_hidro2+1

In [6]:
# Funcion para obtener los componentes especificos del balance:
def f_zone(x,zone):
    df_1 = df.loc[(time,[x]), zone]
    df_2 = df_1.values.tolist()
    df_3 = []
    for i in range(len(perlen)):
        df_3.append(df_2[i]*perlen[i])
    
    df_4 = []
    v = int(len(perlen)/12) # Cada 12 meses
    for i in range(v):
        df_4.append(sum(df_3[12*i:12*(i+1)])/1000000)
    return df_4

In [7]:
# Funcion para calcular el delta del balance:
def f_delta(entrada,salida, zone):
    z_entrada = f_zone(entrada,zone)
    z_salida = f_zone(salida,zone)
    z_delta = []
    for i in range(int(len(perlen)/12)):
        z_delta.append(z_entrada[i]-z_salida[i])
    return z_delta

In [8]:
# Funcion para conocer la tipologia del rio:
def f_tipologia(entrada,salida, zone):
    z_entrada = f_zone(entrada,zone)
    z_salida = f_zone(salida,zone)
    z_delta = []
    z_tipologia = []
    for i in range(int(len(perlen)/12)):
        z_delta.append(z_entrada[i]-z_salida[i])
    for i in range(int(len(perlen)/12)):
        if z_delta[i]>0:
            z_tipologia.append('Perdedor')
        else:
            z_tipologia.append('Ganador')
    return z_tipologia

### *2.1. ZONE 01 - Borde Mira (hm3/año)* 

In [9]:
zone_1 =['Zone 01 - Borde Mira']*int(len(perlen)/12)

In [10]:
lista_z1 = list(zip(Years, zone_1, f_zone('FROM_HEAD_DEP_BOUNDS','ZONE_1'),
                    f_zone('TO_HEAD_DEP_BOUNDS','ZONE_1'),
                    f_delta('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_1')))
ship_columns_z1 = ['Year','Zone','Entrada (hm3)','Salida (hm3)','Delta (hm3)']

In [11]:
ZB_1 = pd.DataFrame(columns=ship_columns_z1, data=lista_z1)

### *2.2. ZONE 02 - Borde Buñol Cheste (hm3/año)* 

In [12]:
zone_2 =['Zone 02 - Borde Buñol Cheste']*int(len(perlen)/12)

In [13]:
lista_z2 = list(zip(Years, zone_2, f_zone('FROM_HEAD_DEP_BOUNDS','ZONE_2'),
                    f_zone('TO_HEAD_DEP_BOUNDS','ZONE_2'),
                    f_delta('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_2')))
ship_columns_z2 = ['Year','Zone','Entrada (hm3)','Salida (hm3)','Delta (hm3)']

In [14]:
ZB_2 = pd.DataFrame(columns=ship_columns_z2, data=lista_z2)

### *2.3. ZONE 03 - Rambla de la Torre (hm3/año)* 

In [15]:
zone_3 =['Zone 03 - Rambla de la Torre']*int(len(perlen)/12)

In [16]:
lista_z3 = list(zip(Years, zone_3, f_zone('FROM_RIVER_LEAKAGE','ZONE_3'),
                    f_zone('TO_RIVER_LEAKAGE','ZONE_3'),
                    f_delta('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_3'),
                   f_tipologia('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_3')))
ship_columns_z3 = ['Year','Zone','Entrada (hm3)','Salida (hm3)',
                   'Delta (hm3)', 'Tipología del río']

In [17]:
ZB_3 = pd.DataFrame(columns=ship_columns_z3, data=lista_z3)

### *2.4. ZONE 04 - Río Madre (hm3/año)* 

In [18]:
zone_4 =['Zone 04 - Río Madre']*int(len(perlen)/12)

In [19]:
lista_z4 = list(zip(Years, zone_4, f_zone('FROM_RIVER_LEAKAGE','ZONE_4'),
                    f_zone('TO_RIVER_LEAKAGE','ZONE_4'),
                    f_delta('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_4'),
                   f_tipologia('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_4')))
ship_columns_z4 = ['Year','Zone','Entrada (hm3)','Salida (hm3)',
                   'Delta (hm3)', 'Tipología del río']

In [20]:
ZB_4 = pd.DataFrame(columns=ship_columns_z4, data=lista_z4)

### *2.5. ZONE 05 - Río Magro (hm3/año)* 

In [21]:
zone_5 =['Zone 05 - Río Magro']*int(len(perlen)/12)

In [22]:
lista_z5 = list(zip(Years, zone_5, f_zone('FROM_RIVER_LEAKAGE','ZONE_5'),
                    f_zone('TO_RIVER_LEAKAGE','ZONE_5'),
                    f_delta('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_5'),
                   f_tipologia('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_5')))
ship_columns_z5 = ['Year','Zone','Entrada (hm3)','Salida (hm3)',
                   'Delta (hm3)','Tipología del río']

In [23]:
ZB_5 = pd.DataFrame(columns=ship_columns_z5, data=lista_z5)

### *2.6. ZONE 06 - Río Buñol (hm3/año)* 

In [24]:
zone_6 =['Zone 06 - Río Buñol']*int(len(perlen)/12)

In [25]:
lista_z6 = list(zip(Years, zone_6, f_zone('FROM_RIVER_LEAKAGE','ZONE_6'),
                    f_zone('TO_RIVER_LEAKAGE','ZONE_6'),
                    f_delta('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_6'),
                   f_tipologia('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_6')))
ship_columns_z6 = ['Year','Zone','Entrada (hm3)','Salida (hm3)',
                   'Delta (hm3)','Tipología del río']

In [26]:
ZB_6 = pd.DataFrame(columns=ship_columns_z6, data=lista_z6)

## *3. RESUMEN* 

In [27]:
# Funcion para los promedios anuales:
def f_prom(entrada,salida, zone):
    z_entrada = f_zone(entrada,zone)
    z_salida = f_zone(salida,zone)
    z_entrada_prom = sum(z_entrada)/len(z_entrada)
    z_salida_prom = sum(z_salida)/len(z_salida)
    z_delta_prom = z_entrada_prom - z_salida_prom

    return [z_entrada_prom, z_salida_prom, z_delta_prom]

In [28]:
prom = [f_prom('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_1'),
        f_prom('FROM_HEAD_DEP_BOUNDS','TO_HEAD_DEP_BOUNDS','ZONE_2'),
       f_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_3'),
       f_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_4'),
       f_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_5'),
       f_prom('FROM_RIVER_LEAKAGE','TO_RIVER_LEAKAGE','ZONE_6')]

In [29]:
# Valores redondeados a 02 decimales:
entrada_prom = []
for i in range(len(prom)):
    entrada_prom.append(round((prom[i][0]),2))

salida_prom = []
for i in range(len(prom)):
    salida_prom.append(round((prom[i][1]),2))

delta_prom = []
for i in range(len(prom)):
    delta_prom.append(round((prom[i][2]),2))

tipo_prom = []
for i in range(len(prom)):
    if i<2:
        tipo_prom.append('')
    else: 
        if delta_prom[i]>0:
            tipo_prom.append('Perdedor')
        else:
            tipo_prom.append('Ganador')

In [30]:
zone_prom = ['1','2','3','4','5','6']
descr_prom = ['Borde Mira', 'Borde Buñol Cheste', 'Rambla de la Torre',
              'Río Madre', 'Río Magro','Río Buñol']

In [31]:
lista_prom = list(zip(zone_prom, descr_prom,entrada_prom, salida_prom, delta_prom, tipo_prom))
ship_columns_prom = ['Zona', 'Descripción','Entrada (hm3)','Salida (hm3)','Delta (hm3)', 'Tipología del río']

In [32]:
ZB_resumen = pd.DataFrame(columns=ship_columns_prom, data=lista_prom)

## ZONEBUDGET - Exportando a Excel

In [33]:
writer_ZB = pd.ExcelWriter(model_ws + '/' + zb_name +'_Total.xlsx')
ZB_resumen.to_excel(writer_ZB, 'PromedioAnual', index=False)
df.to_excel(writer_ZB, 'Total_m3-d', index=True)
ZB_1.to_excel(writer_ZB, 'Zone1', index=False)
ZB_2.to_excel(writer_ZB, 'Zone2', index=False)
ZB_3.to_excel(writer_ZB, 'Zone3', index=False)
ZB_4.to_excel(writer_ZB, 'Zone4', index=False)
ZB_5.to_excel(writer_ZB, 'Zone5', index=False)
ZB_6.to_excel(writer_ZB, 'Zone6', index=False)
writer_ZB.save()